### Install dependencies

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"
#r "nuget:Microsoft.ML.OnnxRuntime,1.9.0"
#r "nuget:Microsoft.ML.ImageAnalytics,1.6.0"
#r "nuget:Microsoft.ML.OnnxTransformer,1.6.0"


Installed Packages Microsoft.ML, 1.6.0 Microsoft.ML.ImageAnalytics, 1.6.0 Microsoft.ML.OnnxRuntime, 1.9.0 Microsoft.ML.OnnxTransformer, 1.6.0

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Image;
using Newtonsoft.Json;
using System;
using System.Collections.Generic;
using System.Drawing;
using System.IO;
using System.Linq;

### Define constant

In [ ]:
// path to mask-rcnn onnx model
var OnnxModelPath = "/path/to/onnx/model";

// prediction size
// we just use one image, so it's 1
var prediction = 1;

// image height
var Height = 600;

// image width
var Width = 800;

// path to mask-rcnn onnx model's label json
var LabelJsonPath = "/path/to/label";

// path to example image
var ImagePath = "/path/to/image";

// onnx model input shape
var ShapeDictionary = new Dictionary<string, int[]>()
{
    {"input", new int[] { prediction, 3, Height, Width} },
};

### Create ModelInput/Output class


In [ ]:
class ModelInput
{
    [ColumnName("ImageSource"), ImageType(800, 600)]
    public Bitmap ImageSource { get; set; }
}

class ModelOutput
{
    // shape [1, 4] 
    [ColumnName("boxes")]
    public float[] Boxes { get; set; } = new float[0];

    // shape [1]
    [ColumnName("labels")]
    public long[] Labels { get; set; } = new long[0];

    // shape: [1]
    [ColumnName("scores")]
    public float[] Scores { get; set; } = new float[0];

    // shape: [1, 1, height, width]
    [ColumnName("masks")]
    public float[] Masks { get; set; } = new float[0];
}

### Define pipeline && load dataset && create MLNet model

In [ ]:
var mlContext = new MLContext();
var pipeline = mlContext.Transforms.ResizeImages(imageWidth: Width, imageHeight: Height, outputColumnName: @"input", inputColumnName: @"ImageSource", cropAnchor: ImageResizingEstimator.Anchor.Center, resizing: ImageResizingEstimator.ResizingKind.Fill)
                                    .Append(mlContext.Transforms.ExtractPixels(outputColumnName: @"input", inputColumnName: @"input", colorsToExtract: ImagePixelExtractingEstimator.ColorBits.Rgb, orderOfExtraction: ImagePixelExtractingEstimator.ColorsOrder.ARGB))
                                    .Append(mlContext.Transforms.ApplyOnnxModel(modelFile: OnnxModelPath, shapeDictionary: ShapeDictionary));

Bitmap image = (Bitmap)Bitmap.FromFile(ImagePath);
var input = new ModelInput(){
    ImageSource = image,
};

var dataset = mlContext.Data.LoadFromEnumerable(new ModelInput[] { input });
var model = pipeline.Fit(dataset);

Error: System.IO.IOException: Model file /path/to/onnx/model does not exists.
   at Microsoft.ML.Transforms.Onnx.OnnxTransformer..ctor(IHostEnvironment env, Options options, Byte[] modelBytes)
   at Microsoft.ML.Transforms.Onnx.OnnxTransformer..ctor(IHostEnvironment env, String[] outputColumnNames, String[] inputColumnNames, String modelFile, Nullable`1 gpuDeviceId, Boolean fallbackToCpu, IDictionary`2 shapeDictionary, Int32 recursionLimit)
   at Microsoft.ML.Transforms.Onnx.OnnxScoringEstimator..ctor(IHostEnvironment env, String modelFile, Nullable`1 gpuDeviceId, Boolean fallbackToCpu, IDictionary`2 shapeDictionary, Int32 recursionLimit)
   at Microsoft.ML.OnnxCatalog.ApplyOnnxModel(TransformsCatalog catalog, String modelFile, IDictionary`2 shapeDictionary, Nullable`1 gpuDeviceId, Boolean fallbackToCpu)
   at Submission#14.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsy

### Create prediction engine && predict

In [ ]:
var predictEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model, dataset.Schema);
var output = predictEngine.Predict(input);
JsonConvert.SerializeObject(output);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#15.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)